In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

# Load documents
loader = TextLoader("my_notes.txt")
docs = loader.load()

# Split into chunks
splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)

# Create embeddings and store in FAISS
embeddings = OpenAIEmbeddings(openai_api_key="your_api_key_here")
vectorstore = FAISS.from_documents(chunks, embeddings)

# Build retriever and LLM
retriever = vectorstore.as_retriever()
llm = ChatOpenAI(temperature=0, openai_api_key="your_api_key_here")

# Retrieval QA chain
qa = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

def speculative_rag(question):

    guess_prompt = PromptTemplate.from_template("Extract keyword for: {question}")
    guess_chain = LLMChain(llm=llm, prompt=guess_prompt)
    keyword = guess_chain.run(question)

    docs = [Document(page_content = "Elephant is the largest animal in the land.")]
    db = FAISS.from_documents(docs, embeddings)

    retriever = db.as_retriever(search_kwargs={"k": 1})
    qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

    return qa_chain.run(f"What do you know about {keyword}?")

print(speculative_rag("Who is the largest land animal?"))